In [7]:
import pandas
import numpy
import pygsheets
import datetime
import re
import pytz

In [10]:
#fuel_type = 'Gas'; file_key = '1LuTFPdSx5QmzNyc-mjjfM1GelPZ586yC-UXNAhnybJ4' # dec 2022 ggit file
#fuel_type = 'Oil'; file_key = '1l_OzcGukwm4gUAymFj2nQ_sCHdZpiCK6tBo6VzvKCRk' # may 2023 goit file
#fuel_type = 'NGL'; file_key = '1l_OzcGukwm4gUAymFj2nQ_sCHdZpiCK6tBo6VzvKCRk' # may 2023 goit file
fuel_type = 'Gas'; file_key = '1TKVxKAvEDyKU-5_MAzDavVRAc6jT0FSK7xwQfJJ56ms' # dec 2023 ggit file
#fuel_type = 'Gas'; file_key = '1foPLE6K-uqFlaYgLPAUxzeXfDO5wOOqE7tibNHeqTek' # CURRENT file

# define the excel file to save tables in
current_time = datetime.datetime.now(pytz.timezone('US/Eastern')).strftime("%Y-%m-%d_T%H%M%S")
excel_writer = pandas.ExcelWriter('Global-'+fuel_type+'-Pipelines-SummarySheets-'+current_time+'.xlsx')

In [12]:
credentials_directory = '/Users/baird/Dropbox/_google-api/'
gc = pygsheets.authorize(service_account_env_var='GDRIVE_API_CREDENTIALS')
#spreadsheet = gc.open_by_key('1foPLE6K-uqFlaYgLPAUxzeXfDO5wOOqE7tibNHeqTek')
spreadsheet = gc.open_by_key(file_key)

gas_pipes = spreadsheet.worksheet('title', 'Gas pipelines').get_as_df(start='A3')
oil_pipes = spreadsheet.worksheet('title', 'Oil/NGL pipelines').get_as_df(start='A3')

pipes_df = pandas.concat([oil_pipes, gas_pipes], ignore_index=True)

#get other relevant sheets
country_ratios_df = spreadsheet.worksheet('title', 'Country ratios by pipeline').get_as_df()
owners_df = spreadsheet.worksheet('title', 'Pipeline operators/owners (1/3)').get_as_df(start='A2')
parents_df = spreadsheet.worksheet('title', 'Parent metadata (3/3)').get_as_df(start='A2')

# remove empty cells for pipes, owners
pipes_df = pipes_df[pipes_df['PipelineName']!='']
owners_df = owners_df[owners_df['Wiki']!='']
owners_df.set_index('ProjectID', inplace=True)

In [13]:
country_ratios_df.replace('--', numpy.nan, inplace=True)
owners_df.replace('',numpy.nan,inplace=True)
owners_df.replace('--',numpy.nan,inplace=True)
pipes_df.replace('--',numpy.nan,inplace=True)

In [14]:
region_df_orig = spreadsheet.worksheet('title', 'Country dictionary').get_as_df(start='A2')
region_df_orig.replace('--',numpy.nan,inplace=True)
region_df_orig.dropna(axis=0, how='any', inplace=True)

region_df_touse = region_df_orig.copy()

### create region-specific dataframes for region, country_ratios_df, owners_df

In [15]:
country_ratios_df_touse = country_ratios_df.loc[country_ratios_df['Country'].isin(
                                            region_df_touse['Country'].tolist())]

owners_df_touse = owners_df.loc[
    ~owners_df['Countries'].apply(
        lambda x: set(x.split(', ')).isdisjoint(set(region_df_touse['Country'].tolist()))
    )]

pipes_df_touse = pipes_df.loc[
    ~pipes_df['Countries'].apply(
        lambda x: set(x.split(', ')).isdisjoint(set(region_df_touse['Country'].tolist()))
    )]

AttributeError: 'float' object has no attribute 'split'

In [ ]:
pipes_df_touse = pipes_df_touse.loc[pipes_df_touse.Fuel==fuel_type]
print(fuel_type)

### sum MergedKmByCountry and MergedKmByRegion

In [ ]:
status_list = ['Proposed', 
               'Construction', 
               'Shelved', 
               'Cancelled', 
               'Operating', 
               'Idle', 
               'Mothballed', 
               'Retired']
country_list = sorted(set(region_df_touse.Country.to_list())) #sorted(list(set(country_ratios_df_touse['Country'])))
region_list = sorted(set(region_df_touse.Region.tolist())) #sorted(list(set(country_ratios_df_touse['Region'])))

In [ ]:
excel_status_list = ['Proposed', 
                     'Construction', 
                     'In Development (Proposed + Construction)', 
                     'Shelved', 
                     'Cancelled', 
                     'Operating', 
                     'Idle', 
                     'Mothballed', 
                     'Retired']
excel_status_list_with_countries = ['ParentHQCountry', 'Country']+excel_status_list
excel_status_list_with_hq = ['ParentHQCountry']+excel_status_list

In [ ]:
country_ratios_df_subset = country_ratios_df_touse.copy()[country_ratios_df_touse['Fuel']==fuel_type]

km_by_country = pandas.DataFrame(columns=status_list, index=country_list)
km_by_region = pandas.DataFrame(columns=status_list, index=region_list)

print('===country-level calculations===')
for status in status_list:
    print(status)
    country_ratios_df_subset_status = country_ratios_df_subset[country_ratios_df_subset['Status']==status]
    km_by_country[status] = country_ratios_df_subset_status.groupby('Country')['MergedKmByCountry'].sum()

print('===regional calculations===')
for status in status_list:
    print(status)
    country_ratios_df_subset_status = country_ratios_df_subset[country_ratios_df_subset['Status']==status]
    km_by_region[status] = country_ratios_df_subset_status.groupby('Region')['MergedKmByCountry'].sum()

# fille NaN with 0.0
km_by_region = km_by_region.fillna(0)
km_by_country = km_by_country.fillna(0)

km_by_region['In Development (Proposed + Construction)'] = km_by_region[['Proposed','Construction']].sum(axis=1)
km_by_country['In Development (Proposed + Construction)'] = km_by_country[['Proposed','Construction']].sum(axis=1)

km_by_country = km_by_country[excel_status_list]
km_by_region = km_by_region[excel_status_list]

km_by_region.index.name = 'Region'
km_by_country.index.name = 'Country'

totals_row = km_by_region.sum(axis=0)
km_by_region.loc['Total'] = totals_row

totals_row = km_by_country.sum(axis=0)
km_by_country.loc['Total'] = totals_row

km_by_country = km_by_country.loc[~(km_by_country==0).all(axis=1)]
km_by_region = km_by_region.loc[~(km_by_region==0).all(axis=1)]

km_by_region.replace(0,'',inplace=True)
km_by_country.replace(0,'',inplace=True)

km_by_region.to_excel(excel_writer, 'Kilometers by region')
km_by_country.to_excel(excel_writer, 'Kilometers by country')

In [ ]:
owner_parent_calculations_df = pandas.DataFrame()
parents_list = parents_df.Parent.tolist()

for idx,row in pipes_df_touse.iterrows():
    parent_string = row.Parent
    #print(parent_string)
    parent_list = re.sub(' \[.*?\]', '', parent_string).split('; ') # all entries must have a Owner [%] syntax
    percent_list = [float(i.rstrip('%'))/100. for i in re.findall('\\d+(?:\\.\\d+)?%', parent_string)]

    if parent_list.__len__()!=percent_list.__len__():
        if percent_list==[]:
            percent_list = [1/parent_list.__len__() for i in parent_list]
        else:
            nmissing = parent_list.__len__()-percent_list.__len__()
            # distribute nans evenly
            total = numpy.nansum(percent_list)
            leftover = 1-total
            percent_list += [leftover/nmissing]*nmissing

    # how many countries is this pipeline in?
    country_ratios_df_temp = country_ratios_df.loc[country_ratios_df.ProjectID==row.ProjectID]
    #print(country_ratios_df_temp)
    country_ratios_df_temp_region = country_ratios_df.loc[(country_ratios_df.ProjectID==row.ProjectID)&
                                                          (country_ratios_df.Country.isin(region_df_touse.Country.tolist()))]
    
    # fraction of this pipeline that is within the regional countries we care about
    # this will be 1.0 when you're doing global calculations...
    fraction_in_region = country_ratios_df_temp_region.MergedKmByCountry.sum()/country_ratios_df_temp.MergedKmByCountry.sum()
    
    for p_idx,parent in enumerate(parent_list):
        
        if parent in parents_list:
            parent_hq_country = parents_df.loc[parents_df.Parent==parent]['ParentHQCountry'].values[0]
        else:
            parent_hq_country = 'Unknown'
        
        for country in country_ratios_df_temp_region.Country.tolist():
            merged_km_by_country = country_ratios_df.loc[(country_ratios_df.ProjectID==row.ProjectID)&
                                                         (country_ratios_df.Country==country),'MergedKmByCountry'].values[0]
            owner_parent_calculations_df = pandas.concat([owner_parent_calculations_df, 
                                                          pandas.DataFrame([{'Parent':parent, 'ProjectID':row.ProjectID, 
                                                                             'FractionOwnership':percent_list[p_idx],
                                                                             'LengthMergedKm':row.LengthMergedKm,
                                                                             'FractionInRegion':fraction_in_region,
                                                                             'MergedKmByCountry':merged_km_by_country,
                                                                             'ParentHQCountry':parent_hq_country,
                                                                             'Country':country,
                                                                             'Status':row.Status,
                                                                             'KmOwnershipInCountry':percent_list[p_idx]*merged_km_by_country}])])

owner_parent_calculations_df['KmOwnership'] = owner_parent_calculations_df['FractionOwnership']*owner_parent_calculations_df['LengthMergedKm']
owner_parent_calculations_df['KmOwnershipInRegion'] = owner_parent_calculations_df['FractionOwnership']*owner_parent_calculations_df['LengthMergedKm']*owner_parent_calculations_df['FractionInRegion']



### version that lists individual countries pipelines run in

### version that doesn't list country that pipelines are in...

In [ ]:
owners_km_by_status_df = owner_parent_calculations_df.groupby(['Parent','ParentHQCountry','Status'])[['KmOwnershipInCountry']].sum()

owners_km_by_status_df = owner_parent_calculations_df.groupby(['Parent','ParentHQCountry','Status']).agg(
    {'KmOwnershipInCountry':['sum']}).unstack().droplevel(axis=1, level=[0,1])

owners_km_by_status_df = owners_km_by_status_df.reindex(columns=status_list) # adds any missing status columns
owners_km_by_status_df = owners_km_by_status_df.reset_index().set_index('Parent')
#owners_km_by_status_df = owners_km_by_status_df.reset_index()
owners_km_by_status_df.columns.name = None

owners_km_by_status_df['In Development (Proposed + Construction)'] = \
    owners_km_by_status_df[['Construction','Proposed']].sum(min_count=1, axis=1)

# rearrange the order of the columns for output
owners_km_by_status_df = owners_km_by_status_df[excel_status_list_with_hq]

totals_row = owners_km_by_status_df.sum(axis=0, min_count=0)
owners_km_by_status_df.loc['Total'] = totals_row
owners_km_by_status_df.loc['Total','Country'] = ''
owners_km_by_status_df.loc['Total','ParentHQCountry'] = ''

owners_km_by_status_df.index.name = 'Owner'
owners_km_by_status_df.rename(columns={#'Country':'Pipeline Country',
                                       'ParentHQCountry':'Owner Headquarters Country'},
                             inplace=True)

owners_km_by_status_df = owners_km_by_status_df.replace(numpy.nan, '')
owners_km_by_status_df = owners_km_by_status_df.replace(0, '')



owners_km_by_status_df.to_excel(excel_writer, sheet_name='Pipeline km by company, country')

### pipeline km by start year, typeexcel_status_list_with_hq

In [ ]:
pipes_started_df = pipes_df_touse.copy()
#pipes_started_eu['StartYearLatest'].replace(numpy.nan,'',inplace=True)

pipes_started_df = pipes_started_df[(pipes_started_df['Status'].isin(['Operating'])) &
                              (pipes_started_df['Fuel']==fuel_type)]
pipes_started_df_sum = pipes_started_df.groupby('StartYearEarliest')['LengthMergedKm'].sum()

In [ ]:
km_by_start_year = pandas.DataFrame(index=list(range(1980,2023)), columns=['Pipeline km'])
km_by_start_year.index.name = 'Start year'

km_by_start_year['Pipeline km'] = pipes_started_df_sum
km_by_start_year.replace(numpy.nan,0,inplace=True)

totals_row = km_by_start_year.sum(axis=0)
km_by_start_year.loc['Total'] = totals_row

km_by_start_year.replace(0,'',inplace=True)

km_by_start_year.to_excel(excel_writer, 'Kilometers by start year')

## save excel file

In [ ]:
excel_writer.close()